<a href="https://colab.research.google.com/github/SarkarPriyanshu/Machine-Learning-Models/blob/main/Question_Answer_Agent_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain -q
!pip install pypdf -q # Pdf reader
!pip install huggingface_hub -q # Large Language Model
!pip install rapidocr-onnxruntime -q # Extract images from pdf as text....
!pip install unstructured -q
!pip install "unstructured[all-docs]" -q
!pip install langchain sentence_transformers -q
!pip install pinecone-client -q
!pip install python-poppler -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.2/178.2 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [52]:
import os
import re
import numpy as np

# Retriver
from langchain.document_loaders import PyPDFLoader,DirectoryLoader,TextLoader
# Pre Processor
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Embedding
from langchain.embeddings import HuggingFaceEmbeddings
# LLM Model
from langchain.llms import HuggingFaceHub
#
from langchain.vectorstores import Pinecone
import pinecone
#
from langchain.chains.question_answering import load_qa_chain
#
from langchain.memory import ConversationBufferWindowMemory

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Changing Working Directory and Loading file names

directory = '/content/drive/MyDrive/QueAnsApp'
os.chdir(directory)

In [5]:
# Loading API keys setting enviornment variables

loader = DirectoryLoader(os.getcwd(), glob="**/*.txt", loader_cls=TextLoader)
API_KEYS = [Api_key.split('=')[1] for Api_key in loader.load_and_split()[0].dict()['page_content'].split('\n')]

os.environ['HUGGINGFACEHUB_API_TOKEN'] = API_KEYS[0].strip()
os.environ['PINECONE_API_KEY'] = API_KEYS[1].strip()
os.environ['PINECONE_ENV'] = 'gcp-starter'


In [6]:
# Changing Working Directory and Loading file names

directory = '/content/drive/MyDrive/Books'
os.chdir(directory)
current_directory = os.getcwd()
files_and_folders = os.listdir(current_directory)

In [7]:
loader = PyPDFLoader(files_and_folders[0])
docs = loader.load_and_split()

In [8]:
print(f'Length of Corpus (PDFs) : {len(docs)}')

Length of Corpus (PDFs) : 356


In [9]:
def document_chunk_splitter(document,chunk_size,chunk_overlap):
  text_splitter = RecursiveCharacterTextSplitter(
      # Set a really small chunk size, just to show.
      chunk_size = chunk_size,
      chunk_overlap  = chunk_overlap
  )
  return text_splitter.split_documents(document)

In [10]:
docs = document_chunk_splitter(docs,chunk_size=1000,chunk_overlap=100)

In [11]:
len(docs)

891

In [12]:
embeddings = HuggingFaceEmbeddings()

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [13]:
embedded_doc =  embeddings.embed_documents([doc.page_content for doc in docs])
embedded_doc_length = len(embedded_doc)

In [14]:
embedded_doc_length

891

In [38]:
embedded_doc =  embeddings.embed_documents([doc.page_content for doc in docs[:3]])
embedded_doc_length = len(embedded_doc)

In [43]:
pinecone.init(
    api_key=os.getenv("PINECONE_API_KEY"),  # find at app.pinecone.io
    environment=os.getenv("PINECONE_ENV"),  # next to api key in console
)

index_name = "langchain-demo"

# First, check if our index already exists. If it doesn't, we create it
if index_name not in pinecone.list_indexes():
    # we create a new index
    pinecone.create_index(name=index_name, metric="cosine", dimension=768)
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
index = Pinecone.from_documents(docs[:3], embeddings, index_name=index_name)

In [44]:
def getsimilardoc(query,k=2,score=False):
  if score:
    similar_docs = index.similarity_search_with_score(query,k=2)
  else:
    similar_docs = index.similarity_search(query,k=2)
  return similar_docs

In [46]:
for string in [doc.page_content for doc in docs[:3]]:
  print(string)

Finding Groups in Data
Finding Groups in Data 
An Introduction to Cluster Analysis 
LEONARD KAUFMAN 
Vrije Universiteit Brussel, 
Brussels, Belgium 
PETER J. ROUSSEEUW 
Universitaire Instelling Antwerpen, 
Antwerp, Belgium 
A JOHN WILEY & SONS, INC., PUBLICATION
Copyright 0 1990,2005 by John Wiley I% Sons, Inc. All rights reserved. 
Published by John Wiley & Sons, Inc., Hoboken, New Jersey. 
Published simultaneously in Canada. 
No part of this publication may be reproduced, stored in a retrieval system or transmitted 
in any form or by any means, electronic, mechanical, photocopying, recording, scanning 
or otherwise, except as permitted under Sections 107 or 108 of the 1976 United States 
Copyright Act, without either the prior written permission of the Publisher, or 
authorization through payment of the appropriate per-copy fee to the Copyright 
Clearance Center, 222 Rosewood Drive, Danvers, MA 01923, (978) 750-8400, fax 
(978) 750-4470. Requests to the Publisher for permission shoul

In [47]:
llm=HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":1e-10})

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [55]:
conversation_with_summary = load_qa_chain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=2),
    verbose=True
)

In [ ]:
similardocs = getsimilardoc('Brussels, Belgium ')
answer = conversation_with_summary.run(input_documents=similardocs,question='tell me about An Introduction to Cluster Analysis ')
print(answer)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Finding Groups in Data 
An Introduction to Cluster Analysis 
LEONARD KAUFMAN 
Vrije Universiteit Brussel, 
Brussels, Belgium 
PETER J. ROUSSEEUW 
Universitaire Instelling Antwerpen, 
Antwerp, Belgium 
A JOHN WILEY & SONS, INC., PUBLICATION

Finding Groups in Data

Question: tell me about An Introduction to Cluster Analysis 
Helpful Answer:
